## Customer Behavior & Churn Insights Using SQL

### Libraries

In [3]:
import duckdb
import pandas as pd

### Upload Dataset

In [ ]:
# Dataset was loaded from https://www.kaggle.com/datasets/blastchar/telco-customer-churn?resource=download

In [9]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


### Preparing SQL table

In [15]:
duckdb.sql('CREATE OR REPLACE TABLE telco AS SELECT * FROM df')

In [16]:
duckdb.sql('SELECT * FROM retail LIMIT 5')

┌────────────┬─────────┬───────────────┬─────────┬────────────┬────────┬──────────────┬──────────────────┬─────────────────┬────────────────┬──────────────┬──────────────────┬─────────────┬─────────────┬─────────────────┬────────────────┬──────────────────┬───────────────────────────┬────────────────┬──────────────┬─────────┐
│ customerID │ gender  │ SeniorCitizen │ Partner │ Dependents │ tenure │ PhoneService │  MultipleLines   │ InternetService │ OnlineSecurity │ OnlineBackup │ DeviceProtection │ TechSupport │ StreamingTV │ StreamingMovies │    Contract    │ PaperlessBilling │       PaymentMethod       │ MonthlyCharges │ TotalCharges │  Churn  │
│  varchar   │ varchar │     int64     │ varchar │  varchar   │ int64  │   varchar    │     varchar      │     varchar     │    varchar     │   varchar    │     varchar      │   varchar   │   varchar   │     varchar     │    varchar     │     varchar      │          varchar          │     double     │   varchar    │ varchar │
├────────────┼──

## SQL Queries

### WINDOW FUNCTION — ROW_NUMBER() order by MonthlyCharges

In [18]:
duckdb.sql("""
SELECT 
    customerID,
    gender,
    MonthlyCharges,
    ROW_NUMBER() OVER (PARTITION BY gender ORDER BY MonthlyCharges DESC) AS rank_within_gender
FROM telco
LIMIT 20;
""")

┌────────────┬─────────┬────────────────┬────────────────────┐
│ customerID │ gender  │ MonthlyCharges │ rank_within_gender │
│  varchar   │ varchar │     double     │       int64        │
├────────────┼─────────┼────────────────┼────────────────────┤
│ 7569-NMZYQ │ Female  │         118.75 │                  1 │
│ 8984-HPEMB │ Female  │         118.65 │                  2 │
│ 5989-AXPUC │ Female  │          118.6 │                  3 │
│ 5734-EJKXG │ Female  │          118.6 │                  4 │
│ 3810-DVDQQ │ Female  │          117.6 │                  5 │
│ 9739-JLPQJ │ Female  │          117.5 │                  6 │
│ 2302-ANTDP │ Female  │         117.45 │                  7 │
│ 6904-JLBGY │ Female  │         117.35 │                  8 │
│ 6650-BWFRT │ Female  │         117.15 │                  9 │
│ 1488-PBLJN │ Female  │         116.85 │                 10 │
│ 0017-IUDMW │ Female  │          116.8 │                 11 │
│ 8628-MFKAX │ Female  │         116.75 │              

### WINDOW FUNCTION — LAG() to see the evolution of ternure

In [30]:
duckdb.sql("""
SELECT 
    customerID,
    tenure,
    LAG(tenure) OVER (ORDER BY tenure) AS previous_tenure
FROM telco
LIMIT 20;
""")

┌────────────┬────────┬─────────────────┐
│ customerID │ tenure │ previous_tenure │
│  varchar   │ int64  │      int64      │
├────────────┼────────┼─────────────────┤
│ 2923-ARZLG │      0 │            NULL │
│ 3213-VVOLG │      0 │               0 │
│ 4367-NUYAO │      0 │               0 │
│ 4472-LVYGI │      0 │               0 │
│ 3115-CZMZD │      0 │               0 │
│ 7644-OMVMY │      0 │               0 │
│ 2520-SGTTA │      0 │               0 │
│ 1371-DWPAZ │      0 │               0 │
│ 4075-WKNIU │      0 │               0 │
│ 2775-SEFEE │      0 │               0 │
│ 5709-LVOEQ │      0 │               0 │
│ 2676-ISHSF │      1 │               0 │
│ 3583-EKAPL │      1 │               1 │
│ 0679-IDSTG │      1 │               1 │
│ 9747-DDZOS │      1 │               1 │
│ 1122-JWTJW │      1 │               1 │
│ 1471-GIQKQ │      1 │               1 │
│ 5766-ZJYBB │      1 │               1 │
│ 6653-CBBOM │      1 │               1 │
│ 0723-DRCLG │      1 │           

### WINDOW FUNCTION — Running Average

In [21]:
duckdb.sql("""
SELECT 
    customerID,
    MonthlyCharges,
    AVG(MonthlyCharges) OVER(ORDER BY MonthlyCharges 
        ROWS BETWEEN 5 PRECEDING AND CURRENT ROW) AS running_avg
FROM telco
LIMIT 20;
""")


┌────────────┬────────────────┬────────────────────┐
│ customerID │ MonthlyCharges │    running_avg     │
│  varchar   │     double     │       double       │
├────────────┼────────────────┼────────────────────┤
│ 6823-SIDFQ │          18.25 │              18.25 │
│ 9764-REAFF │           18.4 │             18.325 │
│ 0827-ITJPH │          18.55 │ 18.400000000000002 │
│ 0621-CXBKL │           18.7 │             18.475 │
│ 9945-PSVIP │           18.7 │ 18.520000000000003 │
│ 9426-SXNHE │          18.75 │ 18.558333333333334 │
│ 2967-MXRAV │           18.8 │ 18.650000000000002 │
│ 7473-ZBDSN │           18.8 │ 18.716666666666665 │
│ 3806-YAZOV │           18.8 │ 18.758333333333333 │
│ 7369-TRPFD │           18.8 │             18.775 │
│ 8464-EETCQ │           18.8 │ 18.791666666666664 │
│ 3387-PLKUI │           18.8 │               18.8 │
│ 6508-NJYRO │           18.8 │               18.8 │
│ 8992-CEUEN │          18.85 │ 18.808333333333334 │
│ 0620-XEFWH │          18.85 │ 18.81666666666

### CTE - Avarege MonthlyCharges by contract type

In [31]:
duckdb.sql("""
WITH avg_contract AS (
    SELECT 
        Contract,
        AVG(MonthlyCharges) AS avg_charge
    FROM telco
    GROUP BY 1
)
SELECT * FROM avg_contract;
""")

┌────────────────┬───────────────────┐
│    Contract    │    avg_charge     │
│    varchar     │      double       │
├────────────────┼───────────────────┤
│ One year       │ 65.04860828241674 │
│ Month-to-month │ 66.39849032258037 │
│ Two year       │   60.770412979351 │
└────────────────┴───────────────────┘

In [22]:
duckdb.sql("""
WITH avg_contract AS (
    SELECT 
        Contract,
        AVG(MonthlyCharges) AS avg_charge
    FROM telco
    GROUP BY 1
)
SELECT * FROM avg_contract;
""")

┌────────────────┬───────────────────┐
│    Contract    │    avg_charge     │
│    varchar     │      double       │
├────────────────┼───────────────────┤
│ One year       │ 65.04860828241674 │
│ Month-to-month │ 66.39849032258037 │
│ Two year       │   60.770412979351 │
└────────────────┴───────────────────┘

### CTE Recursive — Countdown

In [32]:
duckdb.sql("""
WITH RECURSIVE counter(n) AS (
    SELECT 1
    UNION ALL
    SELECT n+1 FROM counter WHERE n < 10
)
SELECT * FROM counter;
""")


┌───────┐
│   n   │
│ int32 │
├───────┤
│     1 │
│     2 │
│     3 │
│     4 │
│     5 │
│     6 │
│     7 │
│     8 │
│     9 │
│    10 │
└───────┘

### SUBQUERY — Above average for MonthlyCharges

In [24]:
duckdb.sql("""
SELECT *
FROM telco
WHERE MonthlyCharges > (SELECT AVG(MonthlyCharges) FROM telco)
LIMIT 20;
""")

┌────────────┬─────────┬───────────────┬─────────┬────────────┬────────┬──────────────┬───────────────┬─────────────────┬────────────────┬──────────────┬──────────────────┬─────────────┬─────────────┬─────────────────┬────────────────┬──────────────────┬───────────────────────────┬────────────────┬──────────────┬─────────┐
│ customerID │ gender  │ SeniorCitizen │ Partner │ Dependents │ tenure │ PhoneService │ MultipleLines │ InternetService │ OnlineSecurity │ OnlineBackup │ DeviceProtection │ TechSupport │ StreamingTV │ StreamingMovies │    Contract    │ PaperlessBilling │       PaymentMethod       │ MonthlyCharges │ TotalCharges │  Churn  │
│  varchar   │ varchar │     int64     │ varchar │  varchar   │ int64  │   varchar    │    varchar    │     varchar     │    varchar     │   varchar    │     varchar      │   varchar   │   varchar   │     varchar     │    varchar     │     varchar      │          varchar          │     double     │   varchar    │ varchar │
├────────────┼─────────┼─

### CASE — Monthly spending classification

In [25]:
duckdb.sql("""
SELECT 
    customerID,
    MonthlyCharges,
    CASE 
        WHEN MonthlyCharges >= 90 THEN 'High spender'
        WHEN MonthlyCharges >= 60 THEN 'Mid spender'
        ELSE 'Low spender'
    END AS spending_category
FROM telco
LIMIT 20;
""")

┌────────────┬────────────────┬───────────────────┐
│ customerID │ MonthlyCharges │ spending_category │
│  varchar   │     double     │      varchar      │
├────────────┼────────────────┼───────────────────┤
│ 7590-VHVEG │          29.85 │ Low spender       │
│ 5575-GNVDE │          56.95 │ Low spender       │
│ 3668-QPYBK │          53.85 │ Low spender       │
│ 7795-CFOCW │           42.3 │ Low spender       │
│ 9237-HQITU │           70.7 │ Mid spender       │
│ 9305-CDSKC │          99.65 │ High spender      │
│ 1452-KIOVK │           89.1 │ Mid spender       │
│ 6713-OKOMC │          29.75 │ Low spender       │
│ 7892-POOKP │          104.8 │ High spender      │
│ 6388-TABGU │          56.15 │ Low spender       │
│ 9763-GRSKD │          49.95 │ Low spender       │
│ 7469-LKBCI │          18.95 │ Low spender       │
│ 8091-TTVAX │         100.35 │ High spender      │
│ 0280-XJGEX │          103.7 │ High spender      │
│ 5129-JLPIS │          105.5 │ High spender      │
│ 3655-SNQYZ

### Aggregation + CASE — Churn by spending bracket

In [26]:
duckdb.sql("""
SELECT 
    CASE 
        WHEN MonthlyCharges >= 90 THEN 'High'
        WHEN MonthlyCharges >= 60 THEN 'Medium'
        ELSE 'Low'
    END AS spending_group,
    COUNT(*) AS customers,
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS churned,
    ROUND(
        SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) * 100.0 
        / COUNT(*), 2
    ) AS churn_rate
FROM telco
GROUP BY 1
ORDER BY churn_rate DESC;
""")


┌────────────────┬───────────┬─────────┬────────────┐
│ spending_group │ customers │ churned │ churn_rate │
│    varchar     │   int64   │ int128  │   double   │
├────────────────┼───────────┼─────────┼────────────┤
│ Medium         │      2392 │     807 │      33.74 │
│ High           │      1744 │     573 │      32.86 │
│ Low            │      2907 │     489 │      16.82 │
└────────────────┴───────────┴─────────┴────────────┘